# Demonstration: call shell command & wait for finish
Demonstrate how to launch a shell command and wait for it to finish.

This involves setting a command and receiving two different values (_stdout_ and _stderr_).  An `ophyd.Signal` is for setting and reading one value.  The `ophyd.Device` can provide the multiple values needed.

## 1. Example shell command
The example shell command is a bash script that executes a 5 second countdown.  The script is shown first:

In [1]:
!cat ./doodle.sh

#!/bin/bash

echo $(date): Doodle demonstration starting
echo $(date): sleep 5 seconds
for i in 5 4 3 2 1; do
    echo $(date): countdown ${i}
    sleep 1
done
echo $(date): Doodle demonstration complete


Now, run it to show how it works.

In [2]:
!bash ./doodle.sh

Fri 26 Mar 2021 10:26:49 AM CDT: Doodle demonstration starting
Fri 26 Mar 2021 10:26:49 AM CDT: sleep 5 seconds
Fri 26 Mar 2021 10:26:49 AM CDT: countdown 5
Fri 26 Mar 2021 10:26:50 AM CDT: countdown 4
Fri 26 Mar 2021 10:26:51 AM CDT: countdown 3
Fri 26 Mar 2021 10:26:52 AM CDT: countdown 2
Fri 26 Mar 2021 10:26:53 AM CDT: countdown 1
Fri 26 Mar 2021 10:26:54 AM CDT: Doodle demonstration complete


## 2. Run from Python `subprocess`

In [3]:
import subprocess
import time

In [4]:
command = "bash ./doodle.sh"

# Start the command
t0 = time.time()
process = subprocess.Popen(
    command,
    shell=True,
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

In [5]:
# wait for the command to finish and collect the outputs.
stdout, stderr = process.communicate()
print(f"{time.time() - t0 = }")

time.time() - t0 = 5.053030014038086


In [6]:
print(f"{stdout = }")
print(f"{stderr = }")

stdout = b'Fri 26 Mar 2021 10:26:54 AM CDT: Doodle demonstration starting\nFri 26 Mar 2021 10:26:54 AM CDT: sleep 5 seconds\nFri 26 Mar 2021 10:26:54 AM CDT: countdown 5\nFri 26 Mar 2021 10:26:55 AM CDT: countdown 4\nFri 26 Mar 2021 10:26:56 AM CDT: countdown 3\nFri 26 Mar 2021 10:26:57 AM CDT: countdown 2\nFri 26 Mar 2021 10:26:58 AM CDT: countdown 1\nFri 26 Mar 2021 10:26:59 AM CDT: Doodle demonstration complete\n'
stderr = b''


In [7]:
# byte strings, must decode to see as string
print(f"{stdout.decode('utf8') = }")
print(f"{stderr.decode('utf8') = }")

stdout.decode('utf8') = 'Fri 26 Mar 2021 10:26:54 AM CDT: Doodle demonstration starting\nFri 26 Mar 2021 10:26:54 AM CDT: sleep 5 seconds\nFri 26 Mar 2021 10:26:54 AM CDT: countdown 5\nFri 26 Mar 2021 10:26:55 AM CDT: countdown 4\nFri 26 Mar 2021 10:26:56 AM CDT: countdown 3\nFri 26 Mar 2021 10:26:57 AM CDT: countdown 2\nFri 26 Mar 2021 10:26:58 AM CDT: countdown 1\nFri 26 Mar 2021 10:26:59 AM CDT: Doodle demonstration complete\n'
stderr.decode('utf8') = ''


## 3. As `ophyd.Signal`

Since this is a demonstration, we show here why the `Signal` implementation just does not provide the right behavior.

An `ophyd.Signal` will be used to accept an input, launch the shell command in a `subprocess` from the `Signal.set()` method, and wait for the response using an `ophyd.Status` object.

Since a redefinition of the `set()` method is needed, it is necessary to create a *subclass* of `ophyd.Signal`.

In [8]:
import ophyd
import threading

class ProcessSignal(ophyd.Signal):

    process = None
    _readback = None
    stderr = None

    def set(self, command, *, timeout=None, settle_time=None):
        st = ophyd.status.Status(self)

        def wait_process():
            self._readback, self.stderr = self.process.communicate(timeout=timeout)
            st._finished()

        self._status = st
        self.process = subprocess.Popen(
            command,
            shell=True,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        # TODO: settle_time
        threading.Thread(target=wait_process, daemon=True).start()
        return st

Create the processor object and run (`.set()`) it.  This will return immediately, before the shell script finishes.  The return result is a `Status` object that `bluesky` will use to wait for the `.set()` operation to finish.

In [9]:
t0 = time.time()
obj = ProcessSignal(name="obj")
st = obj.set("bash ./doodle.sh")
print(f"{st = }")
print(f"{time.time()-t0 = }")

st = Status(obj=ProcessSignal(name='obj', value=0.0, timestamp=1616772420.1250772), done=False, success=False)
time.time()-t0 = 0.008937597274780273


No _timeout_ was not configured.  The shell script runs for 5 seconds so we use the status object to wait for it to complete.

In [10]:
print(f"{obj._status = }")
print(f"{time.time()-t0 = }")
st.wait()
print(f"{obj._status = }")
print(f"{time.time()-t0 = }")

obj._status = Status(obj=ProcessSignal(name='obj', value=0.0, timestamp=1616772420.1250772), done=False, success=False)
time.time()-t0 = 0.031061410903930664
obj._status = Status(obj=ProcessSignal(name='obj', value=b'Fri 26 Mar 2021 10:27:00 AM CDT: Doodle demonstration starting\nFri 26 Mar 2021 10:27:00 AM CDT: sleep 5 seconds\nFri 26 Mar 2021 10:27:00 AM CDT: countdown 5\nFri 26 Mar 2021 10:27:01 AM CDT: countdown 4\nFri 26 Mar 2021 10:27:02 AM CDT: countdown 3\nFri 26 Mar 2021 10:27:03 AM CDT: countdown 2\nFri 26 Mar 2021 10:27:04 AM CDT: countdown 1\nFri 26 Mar 2021 10:27:05 AM CDT: Doodle demonstration complete\n', timestamp=1616772420.1250772), done=True, success=True)
time.time()-t0 = 5.057312250137329


Show what is returned from the `read()` method.

In [11]:
obj.read()

{'obj': {'value': b'Fri 26 Mar 2021 10:27:00 AM CDT: Doodle demonstration starting\nFri 26 Mar 2021 10:27:00 AM CDT: sleep 5 seconds\nFri 26 Mar 2021 10:27:00 AM CDT: countdown 5\nFri 26 Mar 2021 10:27:01 AM CDT: countdown 4\nFri 26 Mar 2021 10:27:02 AM CDT: countdown 3\nFri 26 Mar 2021 10:27:03 AM CDT: countdown 2\nFri 26 Mar 2021 10:27:04 AM CDT: countdown 1\nFri 26 Mar 2021 10:27:05 AM CDT: Doodle demonstration complete\n',
  'timestamp': 1616772420.1250772}}

The problem is seen after we try the `.put()` method

In [12]:
t0 = time.time()
st = obj.put("bash ./doodle.sh")
print(f"{st = }")
print(f"{time.time()-t0 = }")

st = None
time.time()-t0 = 0.0015454292297363281


As before, wait for it to finish and the `value` is still the input command.  Note the `put()` method does not return its status object so we have to use a sleep timer.

In [13]:
print(f"{obj.read() = }")
print(f"{time.time()-t0 = }")
time.sleep(5)
print(f"{obj.read() = }")
print(f"{time.time()-t0 = }")

obj.read() = {'obj': {'value': 'bash ./doodle.sh', 'timestamp': 1616772425.2302892}}
time.time()-t0 = 0.02330780029296875
obj.read() = {'obj': {'value': 'bash ./doodle.sh', 'timestamp': 1616772425.2302892}}
time.time()-t0 = 5.032006025314331


The `ophyd.Signal.put()` method requests the Signal to go to the `value` and then waits for it to get there (that's when it uses up its status object).  The output of the shell script will *never* become the value of the command string.  If we were to set `obj._readback` to be the output from the shell script, then the `put()` method would never return (it hangs because the readback value does not equal the input value).

Signal is not the right interface.

## 4. As `ophyd.Device`

In [14]:
import ophyd
import subprocess
import threading
import time

class ProcessDevice(ophyd.Device):
    command = ophyd.Component(ophyd.Signal, value=None)
    stdout = ophyd.Component(ophyd.Signal, value=None)
    stderr = ophyd.Component(ophyd.Signal, value=None)
    process = None

    def trigger(self):
        """Start acquisition."""
        if self.command.get() is None:
            raise ValueError(f"Must set {self.name}.command.  Cannot be `None`.")
        
        st = ophyd.status.DeviceStatus(self)
        
        def watch_process():
            out, err = self.process.communicate()
            # these are byte strings, decode them to get str
            self.stdout.put(out.decode("utf8"))
            self.stderr.put(err.decode("utf8"))
            self.process = None
            st._finished()

        self._status = st
        self.stderr.put(None)
        self.stdout.put(None)
        self.process = subprocess.Popen(
            self.command.get(),
            shell=True,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        threading.Thread(target=watch_process, daemon=True).start()
        return st

In [15]:
obj = ProcessDevice(name="obj")
obj.stage_sigs["command"] = "bash ./doodle.sh"

In [16]:
t0 = time.time()
obj.stage()
st = obj.trigger()
print(f"{time.time() - t0 = }s")
print(f"{st = }")
obj.read()

time.time() - t0 = 0.008999109268188477s
st = DeviceStatus(device=obj, done=False, success=False)


OrderedDict([('obj_command',
              {'value': 'bash ./doodle.sh', 'timestamp': 1616772430.3297338}),
             ('obj_stdout', {'value': None, 'timestamp': 1616772430.3337305}),
             ('obj_stderr', {'value': None, 'timestamp': 1616772430.3337102})])

In [17]:
st.wait()
print(f"{time.time() - t0 = }s")
print(f"{st = }")
obj.read()

time.time() - t0 = 5.066083669662476s
st = DeviceStatus(device=obj, done=True, success=True)


OrderedDict([('obj_command',
              {'value': 'bash ./doodle.sh', 'timestamp': 1616772430.3297338}),
             ('obj_stdout',
              {'value': 'Fri 26 Mar 2021 10:27:10 AM CDT: Doodle demonstration starting\nFri 26 Mar 2021 10:27:10 AM CDT: sleep 5 seconds\nFri 26 Mar 2021 10:27:10 AM CDT: countdown 5\nFri 26 Mar 2021 10:27:11 AM CDT: countdown 4\nFri 26 Mar 2021 10:27:12 AM CDT: countdown 3\nFri 26 Mar 2021 10:27:13 AM CDT: countdown 2\nFri 26 Mar 2021 10:27:14 AM CDT: countdown 1\nFri 26 Mar 2021 10:27:15 AM CDT: Doodle demonstration complete\n',
               'timestamp': 1616772435.3951068}),
             ('obj_stderr', {'value': '', 'timestamp': 1616772435.3951726})])

In [18]:
obj.unstage()

[ProcessDevice(prefix='', name='obj', read_attrs=['command', 'stdout', 'stderr'], configuration_attrs=[])]

In [19]:
print(obj.stdout.get())

Fri 26 Mar 2021 10:27:10 AM CDT: Doodle demonstration starting
Fri 26 Mar 2021 10:27:10 AM CDT: sleep 5 seconds
Fri 26 Mar 2021 10:27:10 AM CDT: countdown 5
Fri 26 Mar 2021 10:27:11 AM CDT: countdown 4
Fri 26 Mar 2021 10:27:12 AM CDT: countdown 3
Fri 26 Mar 2021 10:27:13 AM CDT: countdown 2
Fri 26 Mar 2021 10:27:14 AM CDT: countdown 1
Fri 26 Mar 2021 10:27:15 AM CDT: Doodle demonstration complete



## 5. Run with bluesky
This is a simplest implementation of the bluesky RunEngine with no custom callbacks, no table output, and no saving data anywhere.  Capture the document stream from `RE` using a simple callback (`document_printer()`) that prints the content of each document.

In this demo, we do not show how to implement a timeout and or interrupt execution of the shell script.

In [20]:
import bluesky
import bluesky.plans as bp
import pprint

def document_printer(key, doc):
    print()
    print(f"***{key}***")
    pprint.pprint(doc)

RE = bluesky.RunEngine({})
RE(bp.count([obj]), document_printer)


***start***
{'detectors': ['obj'],
 'hints': {'dimensions': [(('time',), 'primary')]},
 'num_intervals': 0,
 'num_points': 1,
 'plan_args': {'detectors': ["ProcessDevice(prefix='', name='obj', "
                             "read_attrs=['command', 'stdout', 'stderr'], "
                             'configuration_attrs=[])'],
               'num': 1},
 'plan_name': 'count',
 'plan_type': 'generator',
 'scan_id': 1,
 'time': 1616772435.5289648,
 'uid': '50d238f8-ed4f-4d48-bf46-495f0eb6f5cc',
 'versions': {'bluesky': '1.6.7', 'ophyd': '1.6.1'}}

***descriptor***
{'configuration': {'obj': {'data': {},
                           'data_keys': OrderedDict(),
                           'timestamps': {}}},
 'data_keys': {'obj_command': {'dtype': 'string',
                               'object_name': 'obj',
                               'shape': [],
                               'source': 'SIM:obj_command'},
               'obj_stderr': {'dtype': 'string',
                              'obj

('50d238f8-ed4f-4d48-bf46-495f0eb6f5cc',)

Looks like we got the output from the bash shell script.